In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Batch Processing

여러개의 데이터를 한번의 연산 처리하는 방식  
GPU의 행렬연산을 통해 한번의 메모리업로드를 통해 연산이 가능

기존의 Linear regression with GD를 batch_size번의 데이터마다 업데이트하도록

In [5]:
import pandas as pd
import os

data_path = "/content/drive/MyDrive/랩세미나(new)/tutorial/open_tutorial_DL_NLP/1일차/colab/LR_dataset"
train_file = "train.csv"
test_file = "test.csv"

df_data = pd.read_csv(os.path.join(data_path, train_file)).dropna()
print(df_data)

data_list_x = list(df_data.loc[:,"x"])
data_list_y = list(df_data.loc[:,"y"])
data_num = len(data_list_x)
print(len(data_list_x))
print(len(data_list_y))

        x          y
0    24.0  21.549452
1    50.0  47.464463
2    15.0  17.218656
3    38.0  36.586398
4    87.0  87.288984
..    ...        ...
695  58.0  58.595006
696  93.0  94.625094
697  82.0  88.603770
698  66.0  63.648685
699  97.0  94.975266

[699 rows x 2 columns]
699
699


In [6]:
def lr_hypo(a,b,x):
  return a * x + b

def lr_loss(y, y_pred):
  return (y-y_pred) * (y-y_pred)

In [7]:
def ff_a(a,b,x,y):
  return 2*(a*x+b-y)*x

def ff_b(a,b,x,y):
  return 2*(a*x+b-y)

# from sklearn.preprocessing import RobustScaler
# data_list_x = RobustScaler().fit_transform(np.array(data_list_x).reshape(-1,1)).reshape(-1)
# print(data_list_x)
print(data_list_x)


a = 10
b = -1
lr = 0.0001
batch_size = 20
for e in range(100):
  print("epoch ",e)
  loss_sum = 0
  grad_a_sum = 0
  grad_b_sum = 0
  
  print(a,b)
  for i in range(data_num):
    pred_y = lr_hypo(a,b,data_list_x[i])
    loss_sum += lr_loss(data_list_y[i], pred_y)
    grad_a_sum += ff_a(a,b,data_list_x[i],data_list_y[i])
    grad_b_sum += ff_b(a,b,data_list_x[i],data_list_y[i])

    if (i+1) % batch_size == 0:
      grad_a_mean = grad_a_sum/batch_size
      grad_b_mean = grad_b_sum/batch_size
      loss_mean = loss_sum/batch_size
      a = a - lr * grad_a_mean
      b = b - lr * grad_b_mean
      loss_sum = 0
      grad_a_sum = 0
      grad_b_sum = 0

  print(loss_mean)
  print("\n\n\n")

[24.0, 50.0, 15.0, 38.0, 87.0, 36.0, 12.0, 81.0, 25.0, 5.0, 16.0, 16.0, 24.0, 39.0, 54.0, 60.0, 26.0, 73.0, 29.0, 31.0, 68.0, 87.0, 58.0, 54.0, 84.0, 58.0, 49.0, 20.0, 90.0, 48.0, 4.0, 25.0, 42.0, 0.0, 60.0, 93.0, 39.0, 7.0, 21.0, 68.0, 84.0, 0.0, 58.0, 19.0, 36.0, 19.0, 59.0, 51.0, 19.0, 33.0, 85.0, 44.0, 5.0, 59.0, 14.0, 9.0, 75.0, 69.0, 10.0, 17.0, 58.0, 74.0, 21.0, 51.0, 19.0, 50.0, 24.0, 0.0, 12.0, 75.0, 21.0, 64.0, 5.0, 58.0, 32.0, 41.0, 7.0, 4.0, 5.0, 49.0, 90.0, 3.0, 11.0, 32.0, 83.0, 25.0, 83.0, 26.0, 76.0, 95.0, 53.0, 77.0, 42.0, 25.0, 54.0, 55.0, 0.0, 73.0, 35.0, 86.0, 90.0, 13.0, 46.0, 46.0, 32.0, 8.0, 71.0, 28.0, 24.0, 56.0, 49.0, 79.0, 90.0, 89.0, 41.0, 27.0, 58.0, 26.0, 31.0, 70.0, 71.0, 39.0, 7.0, 48.0, 56.0, 45.0, 41.0, 3.0, 37.0, 24.0, 68.0, 47.0, 27.0, 68.0, 74.0, 95.0, 79.0, 21.0, 95.0, 54.0, 56.0, 80.0, 26.0, 25.0, 8.0, 95.0, 94.0, 54.0, 7.0, 99.0, 36.0, 48.0, 65.0, 42.0, 93.0, 86.0, 26.0, 51.0, 100.0, 94.0, 6.0, 24.0, 75.0, 7.0, 53.0, 73.0, 16.0, 80.0, 77.0, 89.0,

#Linear Regression with Tensor


numpy array tensor를 이용한 batch processing구현  
numpy.ipynb의 텐서 연산 예시와 ppt를 기반으로 구현  

data tensor:  
[[x1, 1],  
 [x2, 1],  
 [x3, 1]]

target tensor:  
[[y1],  
 [y2],  
 [y3]]

param tensor:  
[[a,b]]  

hypo: y = a * x + b * 1   
MSE: (f(x) - y) ^ 2 / n  
MSE의 편미분: 2 * (f(x) - y)f'(x) / n


In [8]:
import numpy as np

def f(param, data):
  return np.dot(data,param.T)

def loss(y, y_pred):
  # print((y_pred - y))
  # print((y_pred - y) * (y_pred - y))
  # print((y_pred - y) * (y_pred - y))
  return np.average((y_pred - y) * (y_pred - y))

def ff_a(param, data, y):
  # print(data)
  # print(data[:,:1])
  # print((f(param, data)-y))
  # print(2*(f(param, data)-y)*data[:,:1])
  return 2*(f(param, data)-y)*data[:,:1]

def ff_b(param, data, y):
  return 2*(f(param, data)-y)

a = 10
b = -1
lr = 0.0001
batch_size = 20
for e in range(100):
  print("epoch ",e)
  print(a,b)
  data_append_list = []
  target_append_list = []
  all_loss = []
  for i in range(data_num):
    data_append_list.append([data_list_x[i],1])
    target_append_list.append([data_list_y[i]])

    if (i+1) % batch_size == 0:
      param_tensor = np.array([[a,b]])
      target_tensor = np.array(target_append_list)
      data_tensor = np.array(data_append_list)
      # print(param_tensor)
      # print(target_tensor)
      # print(data_tensor)

      y_pred = f(param_tensor, data_tensor)
      # print(y_pred)
      batch_loss = loss(target_tensor, y_pred)
      all_loss.append(batch_loss)
      # print(batch_loss)
      grad_a = ff_a(param_tensor, data_tensor, target_tensor)
      grad_b = ff_b(param_tensor, data_tensor, target_tensor)
      # print(grad_a)
      # print(grad_b)

      avg_grad_a = np.average(grad_a)
      avg_grad_b = np.average(grad_b)
      # print(avg_grad_a)
      # print(avg_grad_b)

      a = a - lr * avg_grad_a
      b = b - lr * avg_grad_b

      data_append_list = []
      target_append_list = []

  print(np.average(all_loss))
  print("\n\n\n")
      

epoch  0
10 -1
7822.089683585463




epoch  1
1.0058379536856312 -1.1540273917559567
8.348948996282651




epoch  2
1.0058112621988176 -1.152256107800317
8.3480154030247




epoch  3
1.0057846167041782 -1.1504878759489354
8.347085018638408




epoch  4
1.005758017122464 -1.1487226909427228
8.3461578320847




epoch  5
1.0057314633745622 -1.146960547531651
8.345233832362515




epoch  6
1.0057049553814965 -1.1452014404747404
8.344313008508646




epoch  7
1.0056784930644267 -1.14344536454004
8.343395349597609




epoch  8
1.0056520763446481 -1.141692314504615
8.342480844741534




epoch  9
1.0056257051435922 -1.1399422851545291
8.341569483090021




epoch  10
1.0055993793828253 -1.1381952712848322
8.340661253830007




epoch  11
1.0055730989840492 -1.1364512676995417
8.339756146185657




epoch  12
1.0055468638691005 -1.1347102692116275
8.338854149418209




epoch  13
1.0055206739599507 -1.1329722706429983
8.337955252825873




epoch  14
1.0054945291787054 -1.131237266824485
8.337059445

#Multiple Linear Regression with Tensor


numpy array tensor를 이용한 batch processing구현  
numpy.ipynb의 텐서 연산 예시와 ppt를 기반으로 구현  

data tensor:  
[[x11, x12, 1],  
 [x21, x22, 1],  
 [x31, x32, 1]]

target tensor:  
[[y1],  
 [y2],  
 [y3]]

param tensor:  
[[a,b,c]]  

hypo: y = a * x1 + b * x2 + c   
MSE: (f(x) - y) ^ 2 / n  
MSE의 편미분: 2 * (f(x) - y)f'(x) / n

In [9]:
data_path = "/content/drive/MyDrive/랩세미나(new)/tutorial/open_tutorial_DL_NLP/1일차/colab/LR_dataset"
train_file = "Marketing_Data.csv"

multi_df_data = pd.read_csv(os.path.join(data_path,train_file))
print(multi_df_data)

x_multi_df = multi_df_data.loc[:, ["youtube","facebook","newspaper"]]
print(x_multi_df)
y_sales = list(multi_df_data.loc[:, "sales"])
print(y_sales)

x_youtube = list(x_multi_df.loc[:, "youtube"])
print(x_youtube)
x_facebook = list(x_multi_df.loc[:, "facebook"])
print(x_facebook)
x_newspaper = list(x_multi_df.loc[:, "newspaper"])
print(x_newspaper)

# from sklearn.preprocessing import RobustScaler
# x_multi_df = RobustScaler().fit_transform(x_multi_df)

# print(x_multi_df)

     youtube  facebook  newspaper  sales
0      84.72     19.20      48.96  12.60
1     351.48     33.96      51.84  25.68
2     135.48     20.88      46.32  14.28
3     116.64      1.80      36.00  11.52
4     318.72     24.00       0.36  20.88
..       ...       ...        ...    ...
166    45.84      4.44      16.56   9.12
167   113.04      5.88       9.72  11.64
168   212.40     11.16       7.68  15.36
169   340.32     50.40      79.44  30.60
170   278.52     10.32      10.44  16.08

[171 rows x 4 columns]
     youtube  facebook  newspaper
0      84.72     19.20      48.96
1     351.48     33.96      51.84
2     135.48     20.88      46.32
3     116.64      1.80      36.00
4     318.72     24.00       0.36
..       ...       ...        ...
166    45.84      4.44      16.56
167   113.04      5.88       9.72
168   212.40     11.16       7.68
169   340.32     50.40      79.44
170   278.52     10.32      10.44

[171 rows x 3 columns]
[12.6, 25.68, 14.28, 11.52, 20.88, 11.4, 15.36, 30.4

In [17]:
import numpy as np

def f(param, data):
  return np.dot(data,param.T)

def loss(y, y_pred):
  # print((y_pred - y))
  # print((y_pred - y) * (y_pred - y))
  # print((y_pred - y) * (y_pred - y))
  return np.average((y_pred - y) * (y_pred - y))

def ff_a(param, data, y):
  # print(data)
  # print(data[:,:1])
  # print((f(param, data)-y))
  # print(2*(f(param, data)-y)*data[:,:1])
  return 2*(f(param, data)-y)*data[:,:1]

def ff_b(param, data, y):
  # print(data)
  # print(data[:,:1])
  # print((f(param, data)-y))
  # print(2*(f(param, data)-y)*data[:,:1])
  return 2*(f(param, data)-y)*data[:,1:2]

def ff_c(param, data, y):
  # print(data)
  # print(data[:,:1])
  # print((f(param, data)-y))
  # print(2*(f(param, data)-y)*data[:,:1])
  return 2*(f(param, data)-y)*data[:,2:3]

def ff_d(param, data, y):
  return 2*(f(param, data)-y)

a = 10
b = 3
c = 10
d = 5
lr = 0.00002
batch_size = 20
for e in range(100):
  print("epoch ",e)
  print(a,b)
  data_append_list = []
  target_append_list = []
  all_loss = []
  for i in range(len(x_youtube)):
    data_append_list.append([x_youtube[i], x_facebook[i], x_newspaper[i],1])
    target_append_list.append([y_sales[i]])

    if (i+1) % batch_size == 0:
      param_tensor = np.array([[a,b,c,d]])
      target_tensor = np.array(target_append_list)
      data_tensor = np.array(data_append_list)
      # print(param_tensor)
      # print(target_tensor)
      # print(data_tensor)

      y_pred = f(param_tensor, data_tensor)
      # print(y_pred)
      batch_loss = loss(target_tensor, y_pred)
      all_loss.append(batch_loss)
      # print(batch_loss)
      grad_a = ff_a(param_tensor, data_tensor, target_tensor)
      grad_b = ff_b(param_tensor, data_tensor, target_tensor)
      grad_c = ff_c(param_tensor, data_tensor, target_tensor)
      grad_d = ff_d(param_tensor, data_tensor, target_tensor)
      # print(grad_a)
      # print(grad_b)

      avg_grad_a = np.average(grad_a)
      avg_grad_b = np.average(grad_b)
      avg_grad_c = np.average(grad_c)
      avg_grad_d = np.average(grad_d)
      # print(avg_grad_a)
      # print(avg_grad_b)

      a = a - lr * avg_grad_a
      b = b - lr * avg_grad_b
      c = c - lr * avg_grad_c
      d = d - lr * avg_grad_d

      data_append_list = []
      target_append_list = []

  print(np.average(all_loss))
  print("\n\n\n")
      

epoch  0
10 3
2448888.7504496486




epoch  1
0.2906118241453983 1.3342651168883009
53087.35774428696




epoch  2
0.015503846876287808 0.6268538323614515
23137.342858493637




epoch  3
0.0344501887847537 0.156845135908323
12752.082474786279




epoch  4
0.053905135998572384 -0.1570201755915658
7211.305116202973




epoch  5
0.06688332746328485 -0.35953205395821164
4175.243049351978




epoch  6
0.07501819291464432 -0.4828223965936032
2493.1478516377965




epoch  7
0.07971551168954041 -0.5502213973003827
1547.3541376296962




epoch  8
0.0820024482628626 -0.578640602030692
1004.4492958921384




epoch  9
0.08262264008716365 -0.580303693696149
683.991327473284




epoch  10
0.08211250468426687 -0.564013213825995
487.9711823376091




epoch  11
0.08085731623252504 -0.5360777179687696
362.8662893841913




epoch  12
0.0791322498181222 -0.5009901686661317
279.21383890249433




epoch  13
0.07713240635464821 -0.4619240976940293
220.6052758922488




epoch  14
0.07499476566876871 -0.421096